In [1]:
from obspy.core import Stream, UTCDateTime
from obspy.clients.fdsn import Client

Here is a resource of rotation: [link](https://service.iris.edu/irisws/rotation/docs/1/help/)

Grab some data from our Japan earthquake and look at orientations

In [4]:
client=Client('IRIS')

In [5]:
stime = UTCDateTime('2024-001T07:10.00')
etime = stime + 4*60*60

In [6]:
cat = client.get_events(starttime=stime, endtime=etime, minmagnitude=7)
print(cat[0].origins[0])
print(cat[0].magnitudes[0])

Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=49462969")
	          time: UTCDateTime(2024, 1, 1, 7, 10, 9, 474000)
	     longitude: 137.2638
	      latitude: 37.4893
	         depth: 10000.0
	 creation_info: CreationInfo(author='pt,at,us')
Magnitude
	    resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=214721095")
	            mag: 7.5
	 magnitude_type: 'mww'
	  creation_info: CreationInfo(author='us')


In [8]:
net, sta, loc, chan = 'IU', 'MAJO,ANMO','00', 'LH*'

inv = client.get_stations(network=net, starttime=stime, 
                          endtime=etime, station=sta, 
                          level='response', channel=chan)

In [10]:
st = client.get_waveforms(net, sta, loc, chan, stime, etime)
print(st)

6 Trace(s) in Stream:
IU.ANMO.00.LH1 | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.ANMO.00.LH2 | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.ANMO.00.LHZ | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LH1 | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LH2 | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHZ | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples


Notice that we have LH1 and LH2.  We don't have LHN or LHE.

What are the orientations of these sensors?

In [13]:
print(inv[0][0])
print(inv[0][0][0])
print(inv[0][0][1])

Station ANMO (Albuquerque, New Mexico, USA)
	Station Code: ANMO
	Channel Count: 6/354 (Selected/Total)
	2002-11-19T21:07:00.000000Z - 
	Access: open 
	Latitude: 34.95, Longitude: -106.46, Elevation: 1820.0 m
	Available Channels:
		ANMO.00.LHZ, ANMO.00.LH1, ANMO.00.LH2, ANMO.10.LHZ, ANMO.10.LH1, 
		ANMO.10.LH2
Channel 'LH1', Location '00' 
	Time range: 2018-07-09T20:45:00.000000Z - --
	Latitude: 34.95, Longitude: -106.46, Elevation: 1632.7 m, Local Depth: 188.0 m
	Azimuth: 156.00 degrees from north, clockwise
	Dip: 0.00 degrees down from horizontal
	Channel types: CONTINUOUS, GEOPHYSICAL
	Sampling Rate: 1.00 Hz
	Sensor (Description): None (Streckeisen STS-6A VBB Seismometer)
	Response information available
Channel 'LH2', Location '00' 
	Time range: 2018-07-09T20:45:00.000000Z - --
	Latitude: 34.95, Longitude: -106.46, Elevation: 1632.7 m, Local Depth: 188.0 m
	Azimuth: 246.00 degrees from north, clockwise
	Dip: 0.00 degrees down from horizontal
	Channel types: CONTINUOUS, GEOPHYSICAL
	S

In [14]:
st.rotate("->ZNE", inventory=inv);
print(st)

6 Trace(s) in Stream:
IU.ANMO.00.LHZ | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.ANMO.00.LHN | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.ANMO.00.LHE | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHZ | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHN | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHE | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples


You can find out more about the method here: [link](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.rotate.html)

Finally, if we want to go from NE to radial and transverse we can use our event information.

First we need to get the back azimuth.

In [18]:
from obspy.geodetics.base import gps2dist_azimuth
baz = gps2dist_azimuth(cat[0].origins[0].latitude, cat[0].origins[0].longitude, 
                       inv[0][0].latitude, inv[0][0].longitude)[2]
print(baz)

314.5657100769807


In [19]:
st.select(station='ANMO').rotate("NE->RT",back_azimuth=baz)
print(st)

6 Trace(s) in Stream:
IU.ANMO.00.LHZ | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.ANMO.00.LHR | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.ANMO.00.LHT | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHZ | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHN | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples
IU.MAJO.00.LHE | 2024-01-01T07:10:00.069538Z - 2024-01-01T11:09:59.069538Z | 1.0 Hz, 14400 samples


This is an example of where it is best to do this by station as each station and even with have a different back azimuth.